In [14]:
import tensorflow as tf
import pandas as pd
import matplotlib as plt
import cv2
import os
import glob as gb
import numpy as np
import fnmatch


In [15]:
path = os.getcwd()+ '/Downloads/car_data'

In [16]:
train = path+"/train"
test = path+"/test"
train_labels = []
train_images = []
test_labels = []
test_images = []


In [18]:
train_path =  [folder for folder in os.listdir(train) if not folder.startswith('.')]
test_path =  [folder for folder in os.listdir(test) if not folder.startswith('.')]

size = []
for folder in train_path: 
    files = gb.glob(pathname= str( train +'/' + folder + '/*.jpg'))
    for file in files: 
        image = cv2.imread(file)
        size.append(image.shape)
pd.Series(size).value_counts()





# train_len = -1
# for i in os.walk(train, topdown=True):
#         train_len += len(i[2]) 

# test_len = -1
# for i in os.walk(test, topdown=True):
#         test_len += len(i[2]) 

# train_labels = np.array(len(train_path))
# test_labels = np.array(len(test_path))
# train_images = np.array(train_len)
# test_images = np.array(test_len)




# for folder in train_path:
#     jpg_path = gb.glob(pathname = train + "/" + folder + "/*.jpg")
    
#     for img in jpg_path:
#         img_name = os.path.basename(img)
#         image = cv2.imread(img)
#         np.append(train_labels,np.array(folder))
#         np.append(train_images,np.array(image))

# # for folder in test_path:
# #     jpg_path = gb.glob(pathname = test + "/" + folder + "/*.jpg")
    
# #     for img in jpg_path:
# #         img_name = os.path.basename(img)
# #         image = cv2.imread(img)
# #         np.append(test_labels,np.array(folder))
# #         np.append(test_images,np.array(image))
        
# print(train_labels.shape,train_images.shape)


# print(type(train_images),type(test_images))






        

(480, 640, 3)      989
(768, 1024, 3)     470
(194, 259, 3)      392
(360, 480, 3)      199
(225, 300, 3)      190
                  ... 
(800, 533, 3)        1
(335, 630, 3)        1
(365, 486, 3)        1
(840, 1200, 3)       1
(3240, 4320, 3)      1
Length: 2267, dtype: int64

In [5]:
print()
print(type(train_images),type(test_images))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [6]:
from tensorflow.keras import datasets, layers, models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [7]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [8]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

IndexError: tuple index out of range